## **Mapeamento Esquema Qatar**

##### **___Informações___**

**Projeto:** Qatar

**Autor:** Lucas Pimenta dos Santos Monteiro

**Data:** 17/12/2024

##### **___Objetivo___**

A ideia desse cógido é automatizar a visualização de uma imagem .svg gerada no Inkscape de acordo com o output do programa (report.csv) e os dados de entrada definidos (entry_data.csv).

Para isso foram criados rótulos no Inkscape que identificam cada elemento de texto a partir do inkscape:label. Cada label recebeu o nome equivalente a variável proveniente do report.csv e entry_data.csv que representa o valor desejado.

##### **Importando as Bibliotecas**

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

##### **Lendo o arquivo entry_data.csv e report.csv e armazenando-os em dataframes**

In [2]:
entry_data = pd.read_csv("entry_data.csv")
report = pd.read_csv("report.csv")

##### **Verificando os Dados**

In [3]:
entry_data.head()

,Entry data:,Unnamed: 1,Unnamed: 2
0,Vacuum pressure =,-500.0000000000,mbar
1,Membrane area =,12.9600000000,m²
2,Membrane thickness =,100.0000000000,microns
3,Membrane porosity =,85.0000000000%,NaN
4,Pore diameter =,0.3200000000,microns


In [4]:
report.head()

,Time (s),Desal - Coolant mass flow rate (kg/h),Desal - Hot feedwater inlet temperature (°C),Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Feed membrane temperature (°C),Desal - Membrane gap temperature (°C),Desal - Gap film boundary temperature (°C),Desal - Film wall temperature (°C),...,Turbine - Regenerator cold side outlet temperature (°C),Turbine - Combusiton chamber outlet temperature (°C),Turbine - Turbine outlet temperature (ISENTROPIC) (°C),Turbine - Turbine outlet temperature (REAL) (°C),Turbine - Regenerator hot side outlet temperature (°C),Turbine - Compressor power (W),Turbine - Turbine power (W),Turbine - Net power (W),Turbine - Cycle efficiency,Unnamed: 79
0,126.0,1066.406510,0.303388,42.309038,29.924405,62.538494,57.794801,54.274200,50.493123,47.762324,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
1,252.0,1066.798751,0.300242,42.479936,30.114953,62.557283,57.881810,54.357348,50.572035,47.859690,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
2,378.0,1067.162779,0.298016,42.622330,30.265080,62.564026,57.953585,54.421954,50.628865,47.932474,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
3,504.0,1067.505164,0.296508,42.742125,30.382986,62.561034,58.013252,54.471762,50.668019,47.985607,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN
4,630.0,1067.831308,0.295564,42.844100,30.475288,62.550156,58.063337,54.509794,50.693047,48.023056,...,391.001621,703.641577,311.695918,404.38948,284.198075,-32841.046769,43509.148381,10668.101612,0.23498,NaN


##### **Definindo as Colunas que serão utilizadas de cada dataframe**

In [5]:
userows_entry_data = [
"Reposition water temperature at the entrance of the regulating tank =",
"Cooling heat exchanger flow rate =",
"Temperature at the entrance of the cold side =",
"Cool tower mass flow rate =",
"Mass flow rate at the cold side of the WHRU (line mass flow rate) =",
"WHRU mass flow rate =",
"heat load ="
]

usecols_report = [
"Desal - Hot feedwater outlet temperature (°C)",
"Desal - Cold feedwater inlet temperature (°C)",
"Desal - Cold feedwater outlet temperature (°C)",
"Desal - Hot feedwater outlet salinity (g/L)",
"Desal - Cold feedwater outlet salinity (g/L)",
"Desal - Distilled water production rate (kg/h)",
"Desal - Hot feedwater outlet mass flow rate (kg/h)",
"FeedTank - Water salinity (g/L)",
"FeedTank - Water temperature (°C)",
"ColdHex - Heat transfer rate (W)",
"ColdHex - Hot side inlet temperature (°C)",
"ColdHex - Cold side outlet temperature (°C)",
"HotHex - Heat transfer rate (W)",
"HotHex - Hot side inlet temperature (°C)",
"HotHex - Cold side outlet salinity (g/L)",
"HotHexBypass - Outlet temperature (°C)",
"CoolTower - Heat transfer rate (W)",
"CoolTower - Hot side inlet temperature (°C)",
"CoolTower - Cold side outlet temperature (°C)",
"CoolTowerBypass - mass flow rate (kg/h)",
"WHRU - Heat transfer rate (W)",
"WHRU - Hot side outlet temperature (°C)",
"WHRU - Cold side inlet temperature (°C)",
"WHRU - Cold side outlet temperature (°C)",
"Turbine - Regenerator hot side outlet temperature (°C)",
"Turbine - Net power (W)"
]

##### Filtrando os Dataframes de Acordo com as Colunas

In [6]:
entry_data_filtrado = entry_data[entry_data['Entry data:'].isin(userows_entry_data)]
entry_data_filtrado.head()

,Entry data:,Unnamed: 1,Unnamed: 2
20,Reposition water temperature at the entrance o...,25.0000000000,°C
28,Cooling heat exchanger flow rate =,2000.0000000000,kg/h
29,Temperature at the entrance of the cold side =,30.0000000000,°C
32,Cool tower mass flow rate =,3999.9999960000,kg/h
33,Temperature at the entrance of the cold side =,30.0000000000,°C


In [7]:
report_filtrado = report.filter(items=usecols_report)
report_filtrado.head()

,Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Hot feedwater outlet salinity (g/L),Desal - Cold feedwater outlet salinity (g/L),Desal - Distilled water production rate (kg/h),Desal - Hot feedwater outlet mass flow rate (kg/h),FeedTank - Water salinity (g/L),FeedTank - Water temperature (°C),ColdHex - Heat transfer rate (W),...,CoolTower - Heat transfer rate (W),CoolTower - Hot side inlet temperature (°C),CoolTower - Cold side outlet temperature (°C),CoolTowerBypass - mass flow rate (kg/h),WHRU - Heat transfer rate (W),WHRU - Hot side outlet temperature (°C),WHRU - Cold side inlet temperature (°C),WHRU - Cold side outlet temperature (°C),Turbine - Regenerator hot side outlet temperature (°C),Turbine - Net power (W)
0,42.309038,29.924405,62.538494,206.941561,198.334150,35.193490,1066.406510,201.545399,28.802096,-1179.887551,...,21817.958195,86.876139,34.693906,3019.154264,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
1,42.479936,30.114953,62.557283,208.770661,200.174165,34.801249,1066.798751,203.154539,31.828745,1799.650148,...,21855.731221,86.885121,34.702034,3016.566927,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
2,42.622330,30.265080,62.564026,210.620766,202.029125,34.437221,1067.162779,204.822410,34.234495,4163.273044,...,21880.833685,86.891090,34.707436,3014.838251,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
3,42.742125,30.382986,62.561034,212.491416,203.899996,34.094836,1067.505164,206.543397,36.144233,6035.127267,...,21895.744882,86.894636,34.710644,3013.807881,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
4,42.844100,30.475288,62.550156,214.382175,205.787534,33.768692,1067.831308,208.312126,37.658642,7515.300483,...,21902.445846,86.896229,34.712086,3013.343985,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612


##### Separando apenas o tempo final do output gerado no report.csv

In [8]:
report_filtrado.tail()

,Desal - Hot feedwater outlet temperature (°C),Desal - Cold feedwater inlet temperature (°C),Desal - Cold feedwater outlet temperature (°C),Desal - Hot feedwater outlet salinity (g/L),Desal - Cold feedwater outlet salinity (g/L),Desal - Distilled water production rate (kg/h),Desal - Hot feedwater outlet mass flow rate (kg/h),FeedTank - Water salinity (g/L),FeedTank - Water temperature (°C),ColdHex - Heat transfer rate (W),...,CoolTower - Heat transfer rate (W),CoolTower - Hot side inlet temperature (°C),CoolTower - Cold side outlet temperature (°C),CoolTowerBypass - mass flow rate (kg/h),WHRU - Heat transfer rate (W),WHRU - Hot side outlet temperature (°C),WHRU - Cold side inlet temperature (°C),WHRU - Cold side outlet temperature (°C),Turbine - Regenerator hot side outlet temperature (°C),Turbine - Net power (W)
195,43.595443,30.738533,62.038784,242.126199,233.463396,29.590938,1072.009062,235.723876,42.697402,12283.918256,...,21628.666947,86.831124,34.653174,3031.874183,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
196,43.657030,30.749665,61.986057,244.546277,235.894119,29.205923,1072.394077,238.142612,42.960273,12523.122903,...,21596.302230,86.823426,34.646210,3034.008881,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
197,43.734718,30.759240,61.914910,247.692903,239.058345,28.701735,1072.898265,241.299264,43.219888,12754.138253,...,21551.560679,86.812785,34.636582,3036.941067,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
198,43.843886,30.766771,61.808778,252.200796,243.599872,27.969033,1073.630967,245.839988,43.486984,12983.198536,...,21483.174148,86.796519,34.621867,3041.381087,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612
199,44.039853,30.771100,61.608682,260.269394,251.756662,26.617620,1074.982380,254.009274,43.807479,13240.356314,...,21350.545795,86.764971,34.593328,3049.851096,25009.931042,100.4234,81.666072,87.623599,284.198075,10668.101612


In [9]:
report_filtrado = report_filtrado.iloc[-1]
print(report_filtrado)

Desal - Hot feedwater outlet temperature (°C)                44.039853
Desal - Cold feedwater inlet temperature (°C)                30.771100
Desal - Cold feedwater outlet temperature (°C)               61.608682
Desal - Hot feedwater outlet salinity (g/L)                 260.269394
Desal - Cold feedwater outlet salinity (g/L)                251.756662
Desal - Distilled water production rate (kg/h)               26.617620
Desal - Hot feedwater outlet mass flow rate (kg/h)         1074.982380
FeedTank - Water salinity (g/L)                             254.009274
FeedTank - Water temperature (°C)                            43.807479
ColdHex - Heat transfer rate (W)                          13240.356314
ColdHex - Hot side inlet temperature (°C)                    43.807479
ColdHex - Cold side outlet temperature (°C)                  35.697282
HotHex - Heat transfer rate (W)                           13581.807808
HotHex - Hot side inlet temperature (°C)                     90.000000
HotHex

##### Lendo o arquivo do Inkscape e definindo uma função de substituição de valores

In [10]:
# Caminho para o arquivo SVG
arquivo_svg = 'Esquema_Qatar_2024.12.17.svg'

# Parse o arquivo SVG
tree = ET.parse(arquivo_svg)
root = tree.getroot()

# Definir namespaces
namespaces = {'svg': 'http://www.w3.org/2000/svg', 'inkscape': 'http://www.inkscape.org/namespaces/inkscape'}

def substituir_texto_por_label(label, novo_texto):
    # Encontre todos os elementos do tipo 'text'
    textos = root.findall('.//svg:text', namespaces)
    
    # Para cada elemento 'text' encontrado
    for texto in textos:
        # Verifique se o elemento 'text' possui o atributo 'inkscape:label'
        if texto.attrib.get('{http://www.inkscape.org/namespaces/inkscape}label') == label:
            # Itere sobre os elementos 'tspan' dentro do elemento 'text'
            for tspan in texto.findall('.//svg:tspan', namespaces):
                # Substitua o texto do 'tspan' pelo novo texto
                tspan.text = novo_texto
            break

##### Substituindo os Valores

In [11]:
for coluna, valor in report_filtrado.items():
    unidade = coluna[coluna.rfind('(') + 1:coluna.rfind(')')]
    valor_novo = str(valor) + unidade
    substituir_texto_por_label(coluna, valor_novo)

In [12]:
for linha in entry_data_filtrado.itertuples():
    label = linha[1].rstrip(' =')
    valor = linha[2]
    unidade = linha[3]
    valor_novo = valor + unidade
    substituir_texto_por_label(label, str(valor_novo))

##### Salvando o arquivo

In [13]:
# Salvar as alterações no arquivo SVG
tree.write('Esquema_Qatar_2024.12.17_modificado.svg')